In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [51]:
df = pd.read_csv("data/medical_insurance_scaled.csv", sep=",")
df.head()

,age,sex,bmi,children,smoker,charges_x,northeast,northwest,southeast,charges_y
0,-1.440016,-1.010535,-0.453197,-0.909856,1.973265,-0.565825,-0.565825,-0.610797,1.763737,16884.92400
1,-1.511283,0.989575,0.509049,-0.080117,-0.506774,-0.565825,-0.565825,1.637205,-0.566978,1725.55230
2,-0.798610,0.989575,0.382826,1.579361,-0.506774,-0.565825,-0.565825,1.637205,-0.566978,4449.46200
3,-0.442273,0.989575,-1.304793,-0.909856,-0.506774,-0.565825,1.767330,-0.610797,-0.566978,21984.47061
4,-0.513541,0.989575,-0.292550,-0.909856,-0.506774,-0.565825,1.767330,-0.610797,-0.566978,3866.85520


In [52]:
df.charges.describe()

AttributeError: 'DataFrame' object has no attribute 'charges'

In [ ]:
# On tente de target (charges) par tranche de 5000 jusqu'à 100000 en les label encodant 
df['charges'] = pd.cut(df['charges'], bins=range(0, 100000, 5000), right=False)
le = LabelEncoder()
df['charges'] = le.fit_transform(df['charges'])

In [ ]:
from sklearn.cluster import KMeans


X = df.drop('charges', axis=1)
y = df['charges']

charges
0     357
1     353
2     268
3      85
4      72
7      54
8      41
5      38
9      31
6      29
12      3
10      2
11      2
13      1
Name: count, dtype: int64